In [2]:
import requests
import json

SOLR_HOST = "localhost" 
SOLR_PORT = 8983
CORE_NAME = "techproducts"

url = f"http://{SOLR_HOST}:{SOLR_PORT}/solr/admin/cores"

params = {
    "action": "CREATE",
    "name": CORE_NAME,  
    "configSet": "_default"
}

print(f"Connecting to: {url}")
response = requests.post(url, params=params)

print(f"\nStatus Code: {response.status_code}")
print("\n--- Response from Solr ---")
print(json.dumps(response.json(), indent=2))

Connecting to: http://localhost:8983/solr/admin/cores

Status Code: 500

--- Response from Solr ---
{
  "responseHeader": {
    "status": 500,
    "QTime": 8
  },
  "error": {
    "metadata": [
      "error-class",
      "org.apache.solr.common.SolrException",
      "root-error-class",
      "org.apache.solr.common.SolrException"
    ],
    "msg": "Core with name 'techproducts' already exists.",
    "trace": "org.apache.solr.common.SolrException: Core with name 'techproducts' already exists.\n\tat org.apache.solr.core.CoreContainer.create(CoreContainer.java:1604)\n\tat org.apache.solr.handler.admin.CoreAdminOperation.lambda$static$0(CoreAdminOperation.java:118)\n\tat org.apache.solr.handler.admin.CoreAdminOperation.execute(CoreAdminOperation.java:408)\n\tat org.apache.solr.handler.admin.CoreAdminHandler$CallInfo.call(CoreAdminHandler.java:378)\n\tat org.apache.solr.handler.admin.CoreAdminHandler.handleRequestBody(CoreAdminHandler.java:235)\n\tat org.apache.solr.handler.RequestHandlerBa

In [3]:
# This cell resets the core. Using it for troubleshooting

params = {
    "action": "UNLOAD",
    "core": CORE_NAME,
    "deleteIndex": "true",        # <-- This deletes the data
    "deleteInstanceDir": "true" # <-- This deletes the core's folder
}

print(f"Attempting to UNLOAD (delete) core: {CORE_NAME}...")
response = requests.post(url, params=params)

print(f"\nStatus Code: {response.status_code}")
print("\n--- Response from Solr ---")
print(json.dumps(response.json(), indent=2))

Attempting to UNLOAD (delete) core: techproducts...

Status Code: 200

--- Response from Solr ---
{
  "responseHeader": {
    "status": 0,
    "QTime": 197
  }
}


In [4]:
# This cell removes the documents inside the core, without touching the schema

url = f"http://localhost:8983/solr/{CORE_NAME}/update?commit=true"

# This payload means "delete all documents"
payload = {
    "delete": {
        "query": "*:*"  # The query for "everything"
    }
}

print(f"Attempting to delete all documents from: {CORE_NAME}...")
response = requests.post(url, json=payload)

print(f"\nStatus Code: {response.status_code}")
print("\n--- Response from Solr ---")
print(json.dumps(response.json(), indent=2))

Attempting to delete all documents from: techproducts...

Status Code: 404

--- Response from Solr ---


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# Creating the Schema
url = f"http://{SOLR_HOST}:{SOLR_PORT}/solr/{CORE_NAME}/schema"

payload = {
  "add-field": [
    {"name": "name", "type": "text_general", "multiValued": False},
    {"name": "cat", "type": "string", "multiValued": True},
    {"name": "manu", "type": "string"},
    {"name": "features", "type": "text_general", "multiValued": True},
    {"name": "weight", "type": "pfloat"},
    {"name": "price", "type": "pfloat"},
    {"name": "popularity", "type": "pint"},
    {"name": "inStock", "type": "boolean", "stored": True},
    {"name": "store", "type": "location"}
  ]
}

response = requests.post(url, json=payload)

print(f"Status Code: {response.status_code}")


Status Code: 200


In [7]:
# Inserting the documents 

url = f"http://{SOLR_HOST}:{SOLR_PORT}/solr/{CORE_NAME}/update"

payload =  [ {
    "id" : "978-0641723445",
    "cat" : ["book","hardcover"],
    "name" : "The Lightning Thief",
    "author" : "Rick Riordan",
    "series_t" : "Percy Jackson and the Olympians",
    "sequence_i" : 1,
    "genre_s" : "fantasy",
    "inStock" : True,
    "price" : 12.50,
    "pages_i" : 384
  }
,
  {
    "id" : "978-1423103349",
    "cat" : ["book","paperback"],
    "name" : "The Sea of Monsters",
    "author" : "Rick Riordan",
    "series_t" : "Percy Jackson and the Olympians",
    "sequence_i" : 2,
    "genre_s" : "fantasy",
    "inStock" : True,
    "price" : 6.49,
    "pages_i" : 304
  }
]

response = requests.post(url, json=payload)

print(f"Status Code: {response.status_code}")

Status Code: 200


In [8]:
# commit the changes
url = f"http://localhost:8983/solr/{CORE_NAME}/config"

payload = {
  "set-property": {
    "updateHandler.autoCommit.maxTime": 15000
  }
}

response = requests.post(url, json=payload)
print(f"Status Code: {response.status_code}")

Status Code: 200


In [10]:
# Query time!

url = f"http://{SOLR_HOST}:{SOLR_PORT}/solr/{CORE_NAME}/select"

params = {
    "q": "name:lightning", 
    "fl": "id,name,price",
    "rows": 10
}

response = requests.get(url, params=params)

search_results = response.json()

print(f"Status Code: {response.status_code}")
print(json.dumps(search_results['response'], indent=2))

Status Code: 200
{
  "numFound": 1,
  "start": 0,
  "numFoundExact": true,
  "docs": [
    {
      "id": "978-0641723445",
      "name": "The Lightning Thief",
      "price": 12.5
    }
  ]
}
